In [1]:
from dask.distributed import Client, progress
import dask.distributed

In [2]:
cluster = dask.distributed.LocalCluster()
client = Client(n_workers=4, threads_per_worker=1, memory_limit='0.3GB')
client

/Users/martinarnold/uva/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51769 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51769/status,
Dashboard: http://127.0.0.1:51769/status,Workers: 4
Total threads: 4,Total memory: 1.12 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51770,Workers: 4
Dashboard: http://127.0.0.1:51769/status,Total threads: 4
Started: Just now,Total memory: 1.12 GiB
Comm: tcp://127.0.0.1:51781,Total threads: 1
Dashboard: http://127.0.0.1:51782/status,Memory: 286.10 MiB
Nanny: tcp://127.0.0.1:51773,


In [3]:
import duckdb

# some DuckDB setup
con = duckdb.connect(database=':memory:')
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

In [4]:
train_data = "'../data/train-[1-8].parquet'"

In [5]:
con.execute(f"""

    select *
    from {train_data}

""").df()

,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,__index_level_0__
0,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True,4
1,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True,7
2,tt0012494,Destiny,Der müde Tod,1921,\N,97,5842.0,True,9
3,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True,25
4,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True,38
...,...,...,...,...,...,...,...,...,...
7954,tt9625664,Trauma Center,None,2019,\N,87,12951.0,False,9966
7955,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False,9981
7956,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False,9982
7957,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,None,2020,\N,111,4144.0,True,9996


In [6]:
film_ids = con.execute(f"""
    select distinct tconst,
        label
    from {train_data}
    """).df()

In [7]:
from sklearn.model_selection import train_test_split

# Split training data in train and test split

ids = film_ids.drop('label', axis=1).values
labels = film_ids.loc[:, 'label'].values

# Small test size because cross-validation (and want to train on more data)
X_train_ids, X_test_ids, y_train, y_test = train_test_split(ids, labels, test_size=0.10, random_state=42, stratify=labels)


In [8]:
import pandas as pd
X_train_ids  = pd.DataFrame(X_train_ids, columns=['tconst'])
X_test_ids = pd.DataFrame(X_test_ids, columns=['label'])

X_train_ids.to_parquet("X_train_ids.parquet")
X_test_ids.to_parquet("X_test_ids.parquet")


In [9]:
ids_train = "'X_train_ids.parquet'"
ids_test = "'X_test_ids.parquet'"

In [10]:
# Load all the data into the duck database
baseline_data_train = con.execute(f"""
    with train as (
        select * 
        from {train_data}
    )
 select tconst, 
        try_cast(startYear as integer) as year,
        try_cast(runtimeMinutes as integer) as runtime, 
        try_cast(numVotes as integer) as votes,
        CASE
           WHEN originalTitle IS NULL THEN 0
           ELSE 1
        END AS ForeignFilm,
        -- Count number of words in title
        LENGTH(primaryTitle) - LENGTH(REPLACE(primaryTitle, ' ', '')) + 1 AS n_words,
        label
    from train
    where train.tconst in (select tconst from {ids_train})

""").df()

In [11]:
# Load all the data into the duck database
baseline_data_test = con.execute(f"""
    with train as (
        select * 
        from {train_data}
    )
    select tconst, 
        try_cast(startYear as integer) as year,
        try_cast(runtimeMinutes as integer) as runtime, 
        try_cast(numVotes as integer) as votes,
        CASE
           WHEN originalTitle IS NULL THEN 0
           ELSE 1
        END AS ForeignFilm,
        -- Count number of words in title
        LENGTH(primaryTitle) - LENGTH(REPLACE(primaryTitle, ' ', '')) + 1 AS n_words,
        label
    from train
    where train.tconst in (select tconst from {ids_test})

""").df()

In [12]:
baseline_data_train_noNAN = baseline_data_train.dropna()

In [13]:
X_train = baseline_data_train_noNAN.drop(['tconst',  'label'], axis=1).values
y_train = baseline_data_train_noNAN.loc[:, 'label'].values

In [14]:
print(X_train.shape)
print(y_train.shape)

(5812, 5)
(5812,)


In [21]:
X_test = baseline_data_test.drop(['tconst',  'label'], axis=1).values
y_test = baseline_data_test.loc[:, 'label'].values

In [15]:
import dask.array as da
big_X = da.concatenate([
    da.from_array(X_train, chunks=X_train.shape)
    for _ in range(10)
])
big_X

dask.array<concatenate, shape=(58120, 5), dtype=float64, chunksize=(5812, 5), chunktype=numpy.ndarray>

In [16]:
big_y = da.concatenate([
    da.from_array(y_train, chunks=y_train.shape)
    for _ in range(10)
])
big_y

dask.array<concatenate, shape=(58120,), dtype=bool, chunksize=(5812,), chunktype=numpy.ndarray>

In [17]:
from xgboost import dask as dxgb

In [18]:
dtrain = dxgb.DaskDMatrix(client, big_X, big_y)


In [19]:
output = dxgb.train(
    client,
    {"verbosity": 2, "tree_method": "hist", "objective": "reg:squarederror"},
    dtrain,
    num_boost_round=4,
    evals=[(dtrain, "train")],
)

[0]	train-rmse:0.45794
[1]	train-rmse:0.43264
[2]	train-rmse:0.41758
[3]	train-rmse:0.40803


[15:42:25] task [xgboost.dask-0]:tcp://127.0.0.1:51781 got new rank 0
[15:42:25] task [xgboost.dask-1]:tcp://127.0.0.1:51786 got new rank 1
[15:42:25] task [xgboost.dask-2]:tcp://127.0.0.1:51784 got new rank 2
[15:42:25] task [xgboost.dask-3]:tcp://127.0.0.1:51785 got new rank 3


In [26]:
X_test = da.from_array(X_test)

In [27]:
y_test = da.from_array(y_test)

In [28]:
preds = dxgb.predict(client, output, X_test)

In [30]:
y_pred = preds.compute()

In [36]:
import numpy as np
rounded_array = np.round(y_pred)
# Replace 0s with False and 1s with True
y_pred_bool = rounded_array.astype(bool)

In [38]:
y_test = baseline_data_test.loc[:, 'label'].values
y_test

array([False,  True,  True, ...,  True, False,  True])

In [40]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_bool)
print(f"Accuracy: {accuracy * 100:.2f}% {accuracy_score(y_test, y_pred_bool) * 100:.2f}%")

Accuracy: 73.05% 73.05%
